In [1]:
import rt_analysis

/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.


In [3]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000)
import shared_utils

import geopandas as gpd
import pandas as pd
import folium 
import branca
import datetime as dt
from siuba import *
from bus_service_utils import better_bus_utils

from rt_analysis import rt_filter_map_plot

ZERO_THIRTY_COLORSCALE = branca.colormap.step.RdYlGn_11.scale(vmin=0, vmax=35)

# 100 Recs - D8

## 1 - Corridor Improvements

In [12]:
speed_dict = {"mean_speed_mph": 16}
trip_dict = {"pct_trips_competitive": 0.00}

corridor_gdf_d8 = ((better_bus_utils.select_transit_routes_corridor_improvements(
    speed_dict = speed_dict,
    trip_dict = trip_dict)
                   )
                   >>filter(_.caltrans_district=="08 - San Bernardino")
                  )                    

In [13]:
corridor_gdf_d8

,calitp_itp_id,route_id,category,geometry,service_date,trips_peak,trips_all_day,route_type,mean_speed_mph,route_group,bus_difference_spread,num_competitive,pct_trips_competitive,bus_difference_mean,bus_multiplier_mean,caltrans_district
183,269,1,intersects_shn,"LINESTRING (-117.59438 33.87950, -117.59427 33...",2022-05-04,75.0,118.0,3,12.861547,long,35.4,0.0,0.000,58.276271,2.565847,08 - San Bernardino
184,269,10,intersects_shn,"LINESTRING (-117.45960 33.91186, -117.45951 33...",2022-05-04,15.0,21.0,3,12.719057,medium,13.2,0.0,0.000,26.171429,1.483810,08 - San Bernardino
185,269,11,intersects_shn,"LINESTRING (-117.26826 33.93895, -117.26811 33...",2022-05-04,12.0,20.0,3,12.721494,short,4.8,0.0,0.000,26.760000,1.857500,08 - San Bernardino
186,269,12,intersects_shn,"LINESTRING (-117.52052 33.89185, -117.52055 33...",2022-05-04,19.0,26.0,3,12.516488,long,18.0,0.0,0.000,45.000000,2.172692,08 - San Bernardino
187,269,14,intersects_shn,"LINESTRING (-117.25234 34.04974, -117.25237 34...",2022-05-04,15.0,23.0,3,14.629189,medium,55.8,1.0,0.043,37.278261,1.888696,08 - San Bernardino
188,269,15,intersects_shn,"LINESTRING (-117.37078 33.97625, -117.37141 33...",2022-05-04,32.0,50.0,3,13.089701,medium,20.4,0.0,0.000,21.528000,1.491800,08 - San Bernardino
189,269,16,intersects_shn,"LINESTRING (-117.26826 33.93895, -117.26670 33...",2022-05-04,63.0,111.0,3,13.727505,short,10.2,1.0,0.009,11.572973,1.535405,08 - San Bernardino
190,269,23,intersects_shn,"LINESTRING (-117.15193 33.52372, -117.15370 33...",2022-05-04,19.0,25.0,3,15.418561,medium,61.8,3.0,0.120,35.976000,1.999600,08 - San Bernardino
191,269,24,intersects_shn,"LINESTRING (-117.10707 33.47908, -117.10722 33...",2022-05-04,15.0,22.0,3,10.889040,medium,27.0,0.0,0.000,35.045455,1.973636,08 - San Bernardino
192,269,30,intersects_shn,"LINESTRING (-117.22945 33.78494, -117.22946 33...",2022-05-04,6.0,10.0,3,13.604556,medium,46.8,1.0,0.100,30.780000,1.640000,08 - San Bernardino


In [14]:
#corridor_gdf >> count(_.caltrans_district)

In [15]:
corridor_d8 = (corridor_d8
               >> select(-_.service_date)
              )

corridor_d8.explore("mean_speed_mph", legend=True, tiles="CartoDB positron", cmap=ZERO_THIRTY_COLORSCALE)

In [5]:
# for each transit operator, generate rtfiltermapper
D8_filtermapper = rt_filter_map_plot.from_gcs(itp_id=269, analysis_date=dt.date(2022,5,4))
D8_filtermapper.rt_trips >> head(5)
# get small df of route_id and route_short_name from rt_trips
# inner join w gdf of slow routes
# pass the route names as a filter parameter
# make maps etc  
# output detailed_map_view, fitered to slow segments (p20 speed below certain cutoff (8-12mph?))

found parquet


,calitp_itp_id,calitp_url_number,service_date,trip_key,trip_id,route_id,direction_id,shape_id,calitp_extracted_at,calitp_deleted_at,route_short_name,route_long_name,route_desc,route_type,median_time,direction,mean_speed_mph,calitp_agency_name
0,269,0,2022-05-04,20373467858498656,1808030,15,0,shp-15-58,2022-01-04,2099-01-01,15,Riverside/Downtown-Merced,None,3,18:06:01,Westbound,12.297201,Riverside Transit Agency
1,269,0,2022-05-04,-5290092726997443108,273030,15,0,shp-15-59,2022-01-04,2099-01-01,15,Riverside/Downtown-Merced,None,3,11:45:51,Westbound,12.746440,Riverside Transit Agency
2,269,0,2022-05-04,5684670834086266506,1312030,15,0,shp-15-59,2022-01-04,2099-01-01,15,Riverside/Downtown-Merced,None,3,15:53:31,Westbound,12.488653,Riverside Transit Agency
3,269,0,2022-05-04,-7229066287274309249,1331030,15,0,shp-15-58,2022-01-04,2099-01-01,15,Riverside/Downtown-Merced,None,3,16:36:57,Westbound,11.909406,Riverside Transit Agency
4,269,0,2022-05-04,-2221390301129503368,802030,15,0,shp-15-59,2022-01-04,2099-01-01,15,Riverside/Downtown-Merced,None,3,10:28:32,Westbound,13.873231,Riverside Transit Agency


## 2 - Hotspot Improvements

In [16]:
speed_dict = {"mean_speed_mph": 16}
trip_dict = {"pct_trips_competitive": 0.00}

hotspot_gdf_d8 = ((better_bus_utils.select_transit_routes_hotspot_improvements(
    speed_dict = speed_dict,
    trip_dict = trip_dict)
                   )
                   >>filter(_.caltrans_district=="08 - San Bernardino")
                  )

In [17]:
hotspot_gdf_d8

,calitp_itp_id,route_id,category,geometry,service_date,trips_peak,trips_all_day,route_type,mean_speed_mph,route_group,bus_difference_spread,num_competitive,pct_trips_competitive,bus_difference_mean,bus_multiplier_mean,caltrans_district
25,269,22,other,"LINESTRING (-117.22945 33.78494, -117.22948 33...",2022-05-04,28.0,37.0,3,16.376371,long,49.8,1.0,0.027,37.978378,1.945676,08 - San Bernardino
26,269,3,intersects_shn,"LINESTRING (-117.55463 33.99468, -117.55481 33...",2022-05-04,15.0,28.0,3,34.147090,medium,18.0,0.0,0.000,41.635714,2.734286,08 - San Bernardino
27,269,41,other,"LINESTRING (-117.29453 33.82983, -117.29153 33...",2022-05-04,11.0,16.0,3,19.728451,short,31.8,1.0,0.062,22.762500,1.925000,08 - San Bernardino
28,269,61,intersects_shn,"LINESTRING (-117.15193 33.52372, -117.15370 33...",2022-05-04,16.0,23.0,3,18.076846,long,9.6,0.0,0.000,46.617391,1.766957,08 - San Bernardino
29,269,74,intersects_shn,"LINESTRING (-116.96957 33.80801, -116.96963 33...",2022-05-04,14.0,19.0,3,18.484056,long,20.4,0.0,0.000,48.473684,1.878421,08 - San Bernardino
30,269,8,intersects_shn,"LINESTRING (-117.35201 33.69981, -117.35205 33...",2022-05-04,18.0,24.0,3,16.533120,long,61.2,2.0,0.083,47.875000,2.172500,08 - San Bernardino
31,269,9,intersects_shn,"LINESTRING (-117.35201 33.69981, -117.35763 33...",2022-05-04,14.0,23.0,3,21.794965,short,7.2,0.0,0.000,10.043478,1.507826,08 - San Bernardino


In [18]:
hotspot_gdf_d8 = (hotspot_gdf_d8
               >> select(-_.service_date)
              )

hotspot_gdf_d8.explore("mean_speed_mph", legend=True, tiles="CartoDB positron", cmap=ZERO_THIRTY_COLORSCALE)